# SAKI SS19  Homework 1 - Armin Roth
## Transaction Classification


In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import time
import datetime
import nltk
from nltk.corpus import stopwords
from nltk.stem.cistem import Cistem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, MinMaxScaler
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, roc_curve, auc,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve
from schwifty import IBAN,BIC

In [15]:
#import dataset
df = pd.read_csv("./Exercise 1 - Transaction Classification - Data Set.csv",sep=';',index_col=0, encoding='utf8',header=0)

In [16]:
# # # # # # # # # # # # # # # # # # # #
# gather some information about dataset
# # # # # # # # # # # # # # # # # # # #

print('shape:')
print(df.shape)
print('\nfeatures and types:')
print(df.dtypes)
print('\nis null:')
print(df.isnull().sum())
print('\nclass names:')
print(pd.unique(df.label))
print('\nclass frequencies:')
print(df.label.value_counts())
print('\nunique values per class:')
for column in df:
    print(column +': '+ str(df[column].nunique()))

#df.describe(include='all')

shape:
(209, 11)

features and types:
Auftragskonto                        float64
Buchungstag                           object
Valutadatum                           object
Buchungstext                          object
Verwendungszweck                      object
Beguenstigter/Zahlungspflichtiger     object
Kontonummer                           object
BLZ                                   object
Betrag                                object
Waehrung                              object
label                                 object
dtype: object

is null:
Auftragskonto                        41
Buchungstag                           0
Valutadatum                           0
Buchungstext                          0
Verwendungszweck                      0
Beguenstigter/Zahlungspflichtiger     0
Kontonummer                           1
BLZ                                   1
Betrag                                0
Waehrung                              0
label                                 0
dty

In [17]:
# # # # # # # # # # # # # # # # # # # #
# prepare Data in general
# # # # # # # # # # # # # # # # # # # #

#fill null Values with "0"
df=df.fillna(0)

#Convert Datatypes
df['Kontonummer'] = df['Kontonummer'].astype('str')
df['BLZ'] = df['BLZ'].astype('str')
df['Buchungstag'] = pd.to_datetime(df['Buchungstag'])
df['Betrag'] = [x.replace(',', '.') for x in df['Betrag']]
df['Betrag'] = df['Betrag'].astype('float')
#df['Auftragskonto'] = df['Auftragskonto'].astype('int')
#df['Valutadatum'] = pd.to_datetime(df['Valutadatum'])

#drop unused columns (low variance or duplicates)
df = df.drop(['Auftragskonto','Waehrung','Valutadatum'],axis=1)

In [18]:
# # # # # # # # # # # # # # # # # # # #
# standardize bankaccount
# # # # # # # # # # # # # # # # # # # #

#convert blz and account number to iban
def prepare_bankaccount(kontonummer,blz):
    if(len(kontonummer) == 0):
        return ''
    if(len(kontonummer) != 22):
        if(len(blz) > 8):
            blz = BIC(blz).country_bank_code
        kontonummer = IBAN.generate('DE', bank_code=blz, account_code=kontonummer)
    return str(kontonummer)

#kontonummer vereinlichen
df['iban'] = df.apply(lambda x: prepare_bankaccount(kontonummer=x['Kontonummer'],blz=x['BLZ']), axis=1)

In [19]:
# # # # # # # # # # # # # # # # # # # #
# prepare text features
# # # # # # # # # # # # # # # # # # # #

#cleanup and tokenize text
def prepare_text(text):
    #convert all text to lower case
    text = text.lower()
    #remove punctuation 
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text = text.translate(table)
    #tokenize the text
    tokens = nltk.word_tokenize(text)
    #remove stop words
    for token in tokens:
        if token in stopwords.words('german'):
            tokens.remove(token)

    return " ".join(tokens)
       
#count vectorize and tfidf transform
def vectorize_transform(prepared_text):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(prepared_text).toarray()
    tfidf_transformer = TfidfTransformer()
    X_tfidf = tfidf_transformer.fit_transform(X).toarray()
    topics = vectorizer.get_feature_names()
    return pd.DataFrame(data=X_tfidf, columns=topics)

#count vectorize
def vectorize(prepared_text):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(prepared_text).toarray()
    topics = vectorizer.get_feature_names()
    return pd.DataFrame(data=X, columns=topics)

#select k best features
def k_best(x,y,k):
    selector = SelectKBest(chi2, k=k)
    x_new = selector.fit_transform(x,y)
    columns = selector.get_support(indices=True)
    new_columns = x.columns[columns]
    return pd.DataFrame(x_new,columns=new_columns)


#tokenize feaures
Verwendungszweck_tokenized = df['Verwendungszweck'].apply(prepare_text)
Buchungstext_tokenized = df['Buchungstext'].apply(prepare_text)

#vecorize/transform feaures
df_verwendungszweck = vectorize_transform(Verwendungszweck_tokenized)
df_buchungstext = vectorize_transform(Buchungstext_tokenized)
df_iban = vectorize(df['iban'])

In [20]:
# # # # # # # # # # # # # # # # # # # #
# prepare betrag
# # # # # # # # # # # # # # # # # # # #

#Normalize values
x = df[['Betrag']].values
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_scaled = min_max_scaler.fit_transform(x)
df_betrag = pd.DataFrame(x_scaled, columns=['Betrag'])

In [21]:
# # # # # # # # # # # # # # # # # # # #
# prepare buchungstag
# # # # # # # # # # # # # # # # # # # #

#df_Buchungstag_weekday=df['Buchungstag'].apply(lambda x: x.weekday())
#df_Buchungstag_day=df['Buchungstag'].apply(lambda x: x.day)

In [22]:
# # # # # # # # # # # # # # # # # # # #
# assambling final test/train dataset 
# # # # # # # # # # # # # # # # # # # #

y = df['label']
df_final = pd.concat([k_best(df_verwendungszweck,y,160),df_iban,df_buchungstext,df_betrag],axis=1)

#drop duplicated columns
df_final = df_final.loc[:,~df_final.columns.duplicated()]
#df_final.shape

In [23]:
# # # # # # # # # # # # # # # # # # # #
# training the classifier and evaluation
# # # # # # # # # # # # # # # # # # # #

X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size=0.3,random_state=12345)
#X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size=0.3,random_state=int(time.time()))

#gausian naive bayes
gnb = GaussianNB()

#Train classifier
gnb.fit(X_train,y_train)

#classify
y_pred = gnb.predict(X_test)
y_pred_proba = gnb.predict_proba(X_test)

#evaluate
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ' + str(accuracy) + '\n')

print('Classification Report: \n')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
unique_label = pd.unique(y_test)
print(pd.DataFrame(confusion_matrix(y_test, y_pred, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pre:{:}'.format(x[:3]) for x in unique_label]))


Accuracy: 0.9047619047619048

Classification Report: 

                  precision    recall  f1-score   support

         finance       0.93      0.93      0.93        14
          income       0.86      1.00      0.92         6
         leisure       0.88      1.00      0.93        21
          living       0.75      0.60      0.67         5
         private       1.00      0.83      0.91         6
standardOfLiving       1.00      0.82      0.90        11

       micro avg       0.90      0.90      0.90        63
       macro avg       0.90      0.86      0.88        63
    weighted avg       0.91      0.90      0.90        63

Confusion Matrix:
                       pre:pri  pre:fin  pre:lei  pre:inc  pre:sta  pre:liv
true:private                 5        1        0        0        0        0
true:finance                 0       13        0        1        0        0
true:leisure                 0        0       21        0        0        0
true:income                  0        0 